In [142]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv
import numpy as np
import re

#### Достанем список всех страниц из папки

In [143]:
pages = os.listdir(os.getcwd() + '/pages')

#### Напишем функцию записи файла в csv

In [144]:
def write_csv(data):
    with open('cian.csv', 'a', encoding='utf-8', newline='') as f: # открывает файл и дописывает в него данные в кодировке UTF-8
        writer = csv.writer(f) 
        writer.writerow((data['title'], data['address'], data['price'], data['price_per_square'])) 

#### Напишем функцию записи нужных данных в словарь

In [310]:
def get_page_data(file):
    data = dict()
    with open(os.getcwd() + '/pages/' + file, 'r', encoding='utf-8') as f: 
        soup = BeautifulSoup(f, 'lxml')

        try:
            data['title'] = soup.find('h1', class_=re.compile('\.*--title--\.*')).text
            print(data['title'])
        except:
            data['title'] = np.nan
        try:
            data['address'] = soup.find('address', class_=re.compile('\.*--address--\.*')).text.replace('На карте', '')
            print(data['address'])
        except:
            data['address'] = np.nan
        try:
            data['price'] = soup.find('span', class_=re.compile('\.*--price_value--\.*')).text[:-1].replace('\xa0', '')
            print(data['price'])
        except:
            data['price'] = np.nan
        try:
            data['price_per_square'] = soup.find('div', class_=re.compile('\.*--price_per_meter--\.*')).text[:-5].replace('\xa0', '')
            print(data['price_per_square'])
        except:
            data['price_per_square'] = np.nan
        try:    
            data['total_square'] = soup.find_all('div', class_=re.compile('\.*--info-value--\.*'))[0].text.split('\xa0')[0]
            print(data['total_square'])
        except:
            data['total_square'] = np.nan
        try:    
            data['live_square'] = soup.find_all('div', class_=re.compile('\.*--info-value--\.*'))[1].text.split('\xa0')[0]
            print(data['live_square'])
        except:
            data['live_square'] = np.nan
        try:    
            data['kitchen'] = soup.find_all('div', class_=re.compile('\.*--info-value--\.*'))[2].text.split('\xa0')[0]
            print(data['kitchen'])
        except:
            data['kitchen'] = np.nan
        try:    
            data['floor'] = soup.find_all('div', class_=re.compile('\.*--info-value--\.*'))[3].text.split(' ')[0]
            print(data['floor'])
        except:
            data['floor'] = np.nan
        try:    
            data['total_floor'] = soup.find_all('div', class_=re.compile('\.*--info-value--\.*'))[3].text.split(' ')[2]
            print(data['total_floor'])
        except:
            data['total_floor'] = np.nan
        write_csv(data)

In [312]:
get_page_data(pages[15])

2-комн. апартаменты, 87 м²
Москва, ЦАО, р-н Тверской, Никольская ул., 10
94000000
1 080 460
87
2 из 5
2015
